In [1]:
#Imports
import os
import tensorflow as tf

import sys
sys.path.insert(0, '..')
from utils.federatedAggregation import FederatedAggregation
from utils.reinforcementLearningHelper import *
from utils.federatedLearningHelper import *

In [2]:
#Setup Environments of selected buildings for training, evaluation, and testing

environments, observation_spec, action_spec  = setup_energymanagement_environments(num_buildings=30, ecoPriority=1, noise_scale=0)

#Check environment setup
print(
    "Batch size:", environments["train"][f"building_1"].batch_size, 
    "/ State Space: {} / Action Space: {}".format(observation_spec.shape[0], action_spec.shape[0]),
    "/ Upper bound: {}".format(round(environments["train"][f"building_1"].action_spec().maximum.item(), 3)),
)

Batch size: 1 / State Space: 40 / Action Space: 1 / Upper bound: 2.3


In [3]:
# Cluster similar buildings (K-Means with DTW)

#Set parameter
num_clusters = 9

clustered_buildings = prosumption_clustered_buildings(num_clusters)
clustered_buildings

{0: array([7], dtype=int64),
 1: array([ 3,  4,  5,  9, 10, 11, 16, 23, 25, 26, 27, 30], dtype=int64),
 2: array([1], dtype=int64),
 3: array([24], dtype=int64),
 4: array([12, 13, 14, 15, 19, 20, 22, 28, 29], dtype=int64),
 5: array([ 6, 17], dtype=int64),
 6: array([8], dtype=int64),
 7: array([ 2, 18], dtype=int64),
 8: array([21], dtype=int64)}

In [4]:
# Setup Agent networks
federated_rounds = 20
batch_size = 128
replay_buffer_capacity = 20000
initial_collect_steps = 2000 
collect_steps_per_iteration = 30 
num_iterations = 5000
eval_interval = num_iterations - 1

In [5]:
csv_name = "FL_SAC_9Cluster_OnlyEmissions_Tab4_v2"

In [6]:
# FEDERATED LEARNING - Initalization Round 0

tf.compat.v1.reset_default_graph()
global_step = tf.compat.v1.train.get_or_create_global_step()

#Initalize a global model for each Cluster of similar buildings
for cluster in range(num_clusters):
        
        # 1. Build global agent per cluster
        first_building_in_cluster = clustered_buildings[cluster][0]
        global_sac_agent, global_eval_policy, global_collect_policy = initialize_sac_agent(
                observation_spec=observation_spec, action_spec=action_spec, global_step=global_step, environments=environments,
                )

        # 2. Initially store weights
        model_dir = os.path.join(os.getcwd(), f"models/sac/cluster_{cluster}/FLround{0}_c{num_clusters}_AvgAgg")
        os.makedirs(model_dir, exist_ok=True)
        
        save_sac_weights(global_sac_agent, model_dir)

In [7]:
#FEDERATED LEARNING - Model training for multiple Rounds

#For each federated round and cluster
for federated_round  in range(federated_rounds):
    for cluster_number, buildings_in_cluster in clustered_buildings.items():

        #Iterate through the buildings per cluster
        print(f"Cluster {cluster_number}: Buildings {buildings_in_cluster} Federated round ---", federated_round+1, f"/ {federated_rounds}")
        local_storage = {
            "actor_weights": [], "critic_weights_1": [], "critic_weights_2": [], "performance_metrics": []
            }
        
        for building_index in buildings_in_cluster:
            
            #0. Reset global step
            tf.compat.v1.reset_default_graph()
            global_step = tf.compat.v1.train.get_or_create_global_step()
            
            #1. Initalize local agent
            local_sac_agent, local_eval_policy, local_collect_policy = initialize_sac_agent(
                observation_spec = observation_spec, action_spec = action_spec,
                global_step = global_step, environments = environments,
                )
            
            #2. Set global weights of this training round to agent (loads the weights of last training)
            model_dir = os.path.join(os.getcwd(), f"models/sac/cluster_{cluster_number}/FLround{federated_round}_c{num_clusters}_AvgAgg")
            local_sac_agent = set_weights_to_sac_agent(local_sac_agent, model_dir)
                        
            #3. Prepare training pipeline: Setup iterator, replay buffer, driver
            local_iterator, local_collect_driver, local_time_step, local_policy_state = setup_rl_training_pipeline(
                local_sac_agent, environments["train"][f"building_{building_index}"], replay_buffer_capacity,
                local_collect_policy, initial_collect_steps, collect_steps_per_iteration, batch_size
                )
            
            #4. Train, evaluate agent and store weights
            local_sac_agent, local_storage = local_agent_training_and_evaluation(
                local_iterator, local_collect_driver, local_time_step, local_policy_state, global_step, 
                local_sac_agent, local_eval_policy, local_storage, building_index, num_iterations, environments, agent_type="sac"
                )           

        # Performe Federated aggregation
        average_actor_weights = FederatedAggregation.federated_weigthed_aggregation(local_storage["actor_weights"], local_storage["performance_metrics"])
        average_critic_weights_1 = FederatedAggregation.federated_weigthed_aggregation(local_storage["critic_weights_1"], local_storage["performance_metrics"]) 
        average_critic_weights_2 = FederatedAggregation.federated_weigthed_aggregation(local_storage["critic_weights_2"], local_storage["performance_metrics"]) 
         
        
        #Save federated weights for next round (Round + 1)
        model_dir = os.path.join(os.getcwd(), f"models/sac/cluster_{cluster_number}/FLround{federated_round+1}_c{num_clusters}_AvgAgg")
        os.makedirs(model_dir, exist_ok=True)
        save_federated_sac_weights(model_dir, average_actor_weights, average_critic_weights_1, average_critic_weights_2)

Cluster 0: Buildings [7] Federated round --- 1 / 20
Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


Return:  -5011.4136
Cluster 1: Buildings [ 3  4  5  9 10 11 16 23 25 26 27 30] Federated round --- 1 / 20


Return:  -194548.53


Return:  -195963.73


Return:  -194992.06


...

Return:  -58214.695
Cluster 6: Buildings [8] Federated round --- 20 / 20


Return:  -49437.566
Cluster 7: Buildings [ 2 18] Federated round --- 20 / 20


Return:  -36987.74


Return:  -18544.402
Cluster 8: Buildings [21] Federated round --- 20 / 20


Return:  -11229.669


In [8]:
# LOCAL REFITTING AND EVALUATION

best_federated_round = 3
num_rounds=3
num_test_iterations = 1

result_df = pd.DataFrame(columns=['Building', 'Total Profit'])

for cluster_number, buildings_in_cluster in clustered_buildings.items():
    for building_index in buildings_in_cluster:
        
        for round in range(num_rounds):
            print("Cluster: ", cluster_number, " - Building: ", building_index, " - round: ", round)
            
            #0. Reset global step
            tf.compat.v1.reset_default_graph()
            global_step = tf.compat.v1.train.get_or_create_global_step()

            #1. Initalize local agent
            tf_sac_agent, eval_policy, collect_policy = initialize_sac_agent(
                observation_spec = observation_spec, action_spec = action_spec,
                global_step = global_step, environments = environments,
                )
            
            #2. Set global weights of this training round to agent (loads the weights of last training)
            model_dir = os.path.join(os.getcwd(), f"models/sac/cluster_{cluster_number}/FLround{best_federated_round}_c{num_clusters}_AvgAgg")
            tf_sac_agent = set_weights_to_sac_agent(tf_sac_agent, model_dir)

            #3. Prepare training pipeline: Setup iterator, replay buffer, driver
            iterator, collect_driver, time_step, policy_state = setup_rl_training_pipeline(
                tf_sac_agent, environments["train"][f"building_{building_index}"], replay_buffer_capacity,
                collect_policy, initial_collect_steps, collect_steps_per_iteration, batch_size
                )
            
            #4. Setup wandb logging
            artifact = initialize_wandb_logging(name=f"{csv_name}_Home{building_index}_rd{round}", num_iterations=num_iterations)
            
            #5. Train, evaluate agent and store weights
            result_df, metrics = agent_training_and_evaluation(global_step, num_test_iterations, collect_driver, 
                time_step, policy_state, iterator, tf_sac_agent, eval_policy, building_index, result_df, eval_interval, environments)
            
            #6. End and log wandb
            end_and_log_wandb(metrics, artifact)

Cluster:  0  - Building:  7  - round:  0


c:\Users\rs1044\Documents\GitHub\Federated-Reinforcement-Learning-for-Battery-Charging\src\notebooks\..\utils\reinforcementLearningHelper.py:371: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat(


Building:  7  - Total Profit:  -811.8213307493767  - Total Emissions:  4043.7593540472235
Cluster:  0  - Building:  7  - round:  1


Building:  7  - Total Profit:  -811.8293560596155  - Total Emissions:  4043.817863315884
Cluster:  0  - Building:  7  - round:  2


Building:  7  - Total Profit:  -811.8377008356711  - Total Emissions:  4043.877388475629
Cluster:  1  - Building:  3  - round:  0


...

Building:  18  - Total Profit:  -1671.3231212672606  - Total Emissions:  6566.555589203497
Cluster:  7  - Building:  18  - round:  2


Building:  18  - Total Profit:  -1671.6079327531136  - Total Emissions:  6569.51562873177
Cluster:  8  - Building:  21  - round:  0


Building:  21  - Total Profit:  -1997.7020803733958  - Total Emissions:  7645.332416684095
Cluster:  8  - Building:  21  - round:  1


Building:  21  - Total Profit:  -1997.7044348968045  - Total Emissions:  7645.350970695032
Cluster:  8  - Building:  21  - round:  2


Building:  21  - Total Profit:  -1997.704073409909  - Total Emissions:  7645.343869512274


In [9]:
# Save results
result_df['Setup'] = csv_name
result_df.index.name = 'Building_nr'
result_df.reset_index(inplace=True, drop=True)
os.makedirs('results', exist_ok=True)
result_df.to_csv(f'results/{csv_name}.csv', index=False)

print("Final result: ", result_df["Total Profit"].sum())

Final result:  -106000.24826380856


In [10]:
import requests

def send_telegram_message(bot_token, chat_id, message):
    """Send a message to a Telegram chat via the Bot API."""
    url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
    payload = {
        "chat_id": chat_id,
        "text": message,
        "parse_mode": "Markdown"
    }
    response = requests.post(url, json=payload)
    return response.json()

# Use the function
bot_token = os.getenv('TELEGRAM_BOT_TOKEN')
chat_id = os.getenv('TELEGRAM_CHAT_ID')
message = f"Script SAC FL ist fertig - IP .81!"

result = send_telegram_message(bot_token, chat_id, message)
print(result)

{'ok': True, 'result': {'message_id': 169, 'from': {'id': 7071194232, 'is_bot': True, 'first_name': 'Reinforcement Learning', 'username': 'FederatedRL_Bot'}, 'chat': {'id': 5493937056, 'first_name': 'Jonas', 'last_name': 'Sievers', 'username': 'JonasSievers', 'type': 'private'}, 'date': 1720853085, 'text': 'Script SAC FL ist fertig - IP .81!'}}


In [16]:
result_df['Total Profit'].sum()

-106000.24826380856

In [17]:
std_dev = result_df.groupby('Building')['Total Profit'].std().mean()
std_dev

0.07018748172665101